In [1]:
%matplotlib qt

import numpy as np
import math as m
import matplotlib.pyplot as plt
import os
import h5py
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.interpolate import CubicSpline
from scipy.interpolate import UnivariateSpline
from matplotlib.widgets import Slider


class Particles:
    def __init__(self, x_pos, x_vel, weights):
        self.x = x_pos
        self.vx = x_vel
        self.weights = weights

 
class VariableMatrices:
    def __init__(self, matrices):
        """
        Initialize with a list of 2D matrices. 
        All matrices should have the same shape.
        """
        if not matrices:
            raise ValueError("At least one matrix must be provided.")
        
        # Check if all matrices have the same shape
        matrix_shapes = {matrix.shape for matrix in matrices}
        if len(matrix_shapes) != 1:
            raise ValueError("All matrices must have the same shape.")
        
        self.matrices = np.array(matrices)
        
    def __getitem__(self, index):
        # Access matrices using indexing
        return self.matrices[index]

    def __setitem__(self, index, value):
        # Set matrices using indexing
        if isinstance(value, np.ndarray) and value.shape == self.matrices.shape[1:]:
            self.matrices[index] = value
        else:
            raise ValueError("Value must be a numpy ndarray with the correct shape.")

    def add_matrix(self, new_matrix):
        """
        Add a new matrix to the collection.
        """
        if isinstance(new_matrix, np.ndarray) and new_matrix.shape == self.matrices.shape[1:]:
            self.matrices = np.concatenate((self.matrices, [new_matrix]), axis=0)
        else:
            raise ValueError("New matrix must be a numpy ndarray with the correct shape.")
    
    def __repr__(self):
        return f"VariableMatrices(shape={self.matrices.shape})"


# Function to extract specific variable value from a Python file
def get_in(filename, key_to_find):
    with open(filename, 'r') as file:
        for line in file:
            # Strip any leading/trailing whitespace
            line = line.strip()
            
            # Skip empty lines or lines without '='
            if not line or '=' not in line:
                continue
            
            # Split the line into key and value
            key, value = line.split('=', 1)
            key = key.strip()
            value = value.strip()
            
            # Check if this is the key we are looking for
            if key == key_to_find:
                # Evaluate the value to handle data types (e.g., int, float, string, boolean)
                try:
                    return eval(value)
                except NameError:
                    return value
    
    # Return None if the key was not found
    return None



def Bin_Matrix(particles, x_edges, vx_edges):



    # Initialize the matrix to hold the accumulated weights
    bin_matrix = np.zeros((len(x_edges) - 1, len(vx_edges) - 1))

    # Bin the data
    x_indices = np.digitize(particles.x, x_edges) - 1
    vx_indices = np.digitize(particles.vx, vx_edges) - 1

    # Make sure indices are within the range of bins
    x_indices = np.clip(x_indices, 0, bin_matrix.shape[0] - 1)
    vx_indices = np.clip(vx_indices, 0, bin_matrix.shape[1] - 1)

    # Accumulate the weights in the corresponding bins
    for i in range(len(particles.weights)):
        bin_matrix[x_indices[i], vx_indices[i]] += particles.weights[i]

    return bin_matrix


def h5_extract(file_path):
    h5_file = h5py.File(file_path, 'r')
    particles_data = np.array(h5_file['species_data/particles:data'])
    SpaceDim = h5_file['/Chombo_global'].attrs['SpaceDim']
    numParts = h5_file['/species_data'].attrs['num_particles']
    time = h5_file['/species_data'].attrs['time']
    numPartComps = h5_file['/species_data'].attrs['numPartComps']
    partData = particles_data.reshape((numParts, numPartComps))
    return partData, time


def Phase_space(species,Input_path, x_edges, vx_edges,PPC, nx ):
    if species == 'eon' or species == 'electron' or species == 'electrons':
        folder_path = os.path.join(Input_path, "particle_data/species0_data/")
    elif species == 'ion' or species == 'ions' or species == 'protons':
        folder_path = os.path.join(Input_path, "particle_data/species1_data/")
    else:
        print( 'Choose another species')
        return None


    init_matrix = np.zeros((len(x_edges) - 1, len(vx_edges) - 1))
    INJ_ps = VariableMatrices([init_matrix])
    BG_ps  = VariableMatrices([init_matrix])
    All_ps = VariableMatrices([init_matrix])

    all_files = os.listdir(folder_path)
    h5_files = [f for f in all_files if f.endswith('.h5')]
    h5_files = sorted(h5_files, key=lambda x: int(x.split('parts')[1].split('.h5')[0]))
    
    k=0
    
    
    t= np.zeros( int(np.size(h5_files)) + 1)
    #h5_file = h5py.File(file_path, 'r')
    for file_name in h5_files:
        k +=1
        
        file_path = os.path.join(folder_path, file_name)
        partData, time = h5_extract(file_path)
        weights = partData[:,0]
        x_pos = partData[:,1]
        y_pos = partData[:,2]
        z_pos = partData[:,3]
        x_vel = partData[:,4]
        y_vel = partData[:,5]
        z_vel = partData[:,6]
        ID = partData[:,7]
        
        
    
        All_particles =  Particles(x_pos,x_vel,weights)
        INJ_particles  =  Particles(x_pos[nx*PPC:],x_vel[nx*PPC:],weights[nx*PPC:])
        BG_particles =  Particles(x_pos[:nx*PPC],x_vel[:nx*PPC:],weights[:nx*PPC])
    
        All_matrix = Bin_Matrix(All_particles,x_edges, vx_edges)
        BG_matrix  = Bin_Matrix(BG_particles,x_edges, vx_edges)
        INJ_matrix = Bin_Matrix(INJ_particles,x_edges, vx_edges)
    
        All_ps.add_matrix(All_matrix)
        INJ_ps.add_matrix(INJ_matrix)
        BG_ps.add_matrix(BG_matrix)
        t[k] =+ time
         
    return All_ps, INJ_ps, BG_ps, t









In [5]:
data = 'test_ind'

PIC_path= "../PICNIC/" 
Input_path = os.path.join(PIC_path, data)
input_file = os.path.join(Input_path, str(data +'.in'))
ppc_e = int(get_in(input_file, 'IC.electron.parts_per_cell'))
ppc_i = int(get_in(input_file, 'IC.proton.parts_per_cell'))
nx = np.array(get_in(input_file, 'grid.num_cells'), dtype=float)
Lx =str(get_in(input_file, 'grid.X_max'))
numeric_part = Lx.split('#')[0].strip()
Lx= np.array(numeric_part,dtype=float)
Lx = int(Lx)
nx = int(nx)
dx = Lx/nx



#Let us define the bin matrix
x_edges=  np.linspace(0,Lx,nx+1)
#nvx can be choose arbitrarly, let us start with 100
nvx=500 

vx_edges= np.linspace(-0.001, 0.001, nvx)#np.linspace(-1e-6,1e-6,nvx+1)*3e8  #vx lims are arbitrary



obj, a, b, t= Phase_space('ions',Input_path, x_edges, vx_edges, ppc_e, nx)
#obj = INJ_ps_eon
#obj = BG_ps_eon


In [6]:
# Create a figure and axis for plotting
fig, ax = plt.subplots(figsize=(10, 6))
plt.subplots_adjust(left=0.1, bottom=0.25)  # Adjust plot area for slider

def update(val):
    """
    Update function for the slider.
    """
    idx = int(slider.val)  # Get the slider value as an integer index
    ax.imshow(np.rot90(obj[idx],k=1), cmap='viridis', aspect='auto',extent=[x_edges.min(), x_edges.max(),vx_edges.min()*3e8, 3e8*vx_edges.max()])
    ax.set_title(f"Phase Space - t : {t[idx]}")
    ax.set_xlabel('x [m]')
    ax.set_ylabel('vx [-]')
    fig.canvas.draw_idle()



# Initial display
#fig, ax = plt.subplots(figsize=(10, 6))
#ax.imshow(obj[0], cmap='gist_heat', interpolation='nearest')
ax.set_title("Phase space - time = 0" )
ax.set_xlabel('x [m]')
ax.set_ylabel('vx [-]')
# Add a slider for navigating matrices
ax_slider = plt.axes([0.1, 0.1, 0.65, 0.03], facecolor='lightgoldenrodyellow')
slider = Slider(ax_slider, 'Index', 0, len(obj.matrices) - 1, valinit=0, valfmt='%0.0f')

# Connect the slider to the update function
slider.on_changed(update)

plt.show()